This file is reserved for running BERTopic on bacterial pneumonia to find subgroups associated with severity and clinical outcomes 

Radgraph extracts are expected to have been processsed into sentence-like statements and turned into embeddings 

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

# Custom stopwords for structured format (if radgraph extracts are directly used)
structured_stopwords = [
    'is', 'an', 'anatomy', 'observation', 'modifies', 'located', 'at'
]

In [7]:
pneumonia_type_df = pd.read_csv('../NER_embeddings/bacterial/radgraph_with_embeddings.csv')
pneumonia_type_df

,Unnamed: 0.1,subject_id_x,hadm_id,study_id,dicom_id,pneumonia_type,Severe,Unnamed: 0,subject_id_y,report_path,...,technique,comparison,findings,impression,has_comparison,report_length,radgraph_text,extracts,processed_radgraph,embedding
0,0,11146739,20000057,57595001,d46de6bb-6ee08654-dd5f127c-1c339f58-f780e465,other,False,536,11146739,../../../severity_data/report_files/p11/p11146...,...,NaN,none.,there are coarse interstitial markings bilater...,NaN,True,1130,NaN,"{'0': {'text': 'nan', 'entities': {}, 'data_so...",NaN,"[-0.035868603736162186, -0.13423331081867218, ..."
1,1,18110461,20001947,57106576,1f239460-e00a31a5-81bdb260-f2929be7-f7cb2f7d,bacterial,False,3906,18110461,../../../severity_data/report_files/p18/p18110...,...,chest pa and lateral,[REMOVED],cardiac silhouette size is normal. mediastinal...,findings concerning for multifocal pneumonia. ...,True,645,cardiac silhouette size is normal. mediastinal...,{'0': {'text': 'cardiac silhouette size is nor...,cardiac is an anatomy. silhouette modifies car...,"[0.03332185745239258, -0.005314456298947334, -..."
2,2,18346104,20002712,56389963,33125f0b-b2a6d1a2-1408ce03-33729f30-68e6fd64,other,False,4007,18346104,../../../severity_data/report_files/p18/p18346...,...,single portable ap view of the chest.,chest radiograph from [REMOVED].,"compared with the prior radiograph, no signifi...","compared with the prior radiograph, no change ...",True,693,"compared with the prior radiograph, no signifi...",{'0': {'text': 'compared with the prior radiog...,significant modifies change. change modifies o...,"[0.039139412343502045, -0.016893945634365082, ..."
3,3,19669999,20005479,56328195,927ce6d1-2f7acc58-55042c9a-e688c114-129c4009,other,True,4612,19669999,../../../severity_data/report_files/p19/p19669...,...,NaN,NaN,lung volumes are low. a large hiatal hernia ca...,low lung volumes and large hiatal hernia. new ...,False,712,lung volumes are low. a large hiatal hernia ca...,{'0': {'text': 'lung volumes are low . a large...,lung is an anatomy. volumes modifies lung. low...,"[0.058571215718984604, -0.01945420540869236, -..."
4,4,12351713,20007567,59692077,aa33b0f7-a2e3dc48-68b210fb-fb0a6ea5-bd1bf388,other,False,1138,12351713,../../../severity_data/report_files/p12/p12351...,...,pa and lateral chest radiographs,prior outside chest ct from [REMOVED] and ches...,lung volumes are decreased. there is susbtanti...,low lung volumes with substantial bibasilar at...,True,569,lung volumes are decreased. there is susbtanti...,{'0': {'text': 'lung volumes are decreased . t...,lung is an anatomy. volumes modifies lung. dec...,"[0.027617672458291054, -0.013795840553939342, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4437,4437,10623647,29991969,59289169,f47aa7aa-8461e734-cc03c9ac-f152661b-4700bd0c,bacterial,False,287,10623647,../../../severity_data/report_files/p10/p10623...,...,chest: frontal and lateral,[REMOVED],bilateral patchy pulmonary opacities appear sl...,patchy bilateral mid to lower lung opacities a...,True,1760,bilateral patchy pulmonary opacities appear sl...,{'0': {'text': 'bilateral patchy pulmonary opa...,bilateral modifies pulmonary. patchy modifies ...,"[0.03257104381918907, -0.026108980178833008, -..."
4438,4438,15116068,29993812,55182265,65d1408e-bc59a65c-0be0fa6d-fb54b613-45abd8c1,bacterial,False,2406,15116068,../../../severity_data/report_files/p15/p15116...,...,single portable semi upright radiograph the ch...,"chest ct: [REMOVED], [REMOVED]",extensive bronchiectasis is again noted in the...,"extensive bilateral bronchiectasis, with super...",True,1100,extensive bronchiectasis is again noted in the...,{'0': {'text': 'extensive bronchiectasis is ag...,extensive modifies bronchiectasis. bronchiecta...,"[0.03614714369177818, -0.008272165432572365, -..."
4439,4439,17025867,29996361,50696726,99011231-5f716ee8-5e61eadd-447b48c4-4ec5255a,bacterial,False,3357,17025867,../../../severity_data/report_files/p17/p17025...,...,NaN,radiograph avail

In [31]:
import torch
embeddings = torch.load('../NER_embeddings/bacterial/radgraph_embeddings.pt')

### BERTOPIC - all pneumonia types

In [39]:
from umap import UMAP
import pandas as pd 
from hdbscan import HDBSCAN
from transformers import AutoTokenizer, AutoModel
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
import torch
import os
import numpy as np 
import ast
# Load the embeddings

# Read the DataFrame
pneumonia_type_df = pd.read_csv('../NER_embeddings/bacterial/radgraph_with_embeddings.csv')

# Ensure 'embeddings' column is treated as a list of floats
def convert_to_array(embedding_str):
    try:
        return np.array(ast.literal_eval(embedding_str), dtype=np.float32)
    except (ValueError, SyntaxError):
        return None  # Return None for rows with invalid embeddings

# Convert embeddings column
pneumonia_type_df['embedding'] = pneumonia_type_df['embedding'].apply(convert_to_array)

# Drop rows where embeddings couldn't be converted
pneumonia_type_df = pneumonia_type_df.dropna(subset=['embedding'])

# Convert to proper NumPy array
embeddings = np.vstack(pneumonia_type_df['embedding'].values)

# Ensure embeddings shape is correct
print(f"Embeddings shape: {embeddings.shape}")  # Should be (num_docs, embedding_dim)


# Remove NaN or empty radgraph_text entries
pneumonia_type_df = pneumonia_type_df[pneumonia_type_df['radgraph_text'].notna()]
pneumonia_type_df = pneumonia_type_df[pneumonia_type_df['radgraph_text'].str.strip() != '']

# Extract clean docs and embeddings
docs = pneumonia_type_df['radgraph_text'].astype(str).tolist()
embeddings = np.vstack(pneumonia_type_df['embedding'].values)

# Ensure shape consistency
assert len(docs) == embeddings.shape[0], "Mismatch between docs and embeddings!"

# Load the previously saved CXR model
model_path = "../../models/RADGRAPH_embeddings/bacterial"
tokenizer = AutoTokenizer.from_pretrained(model_path)
embedding_model = AutoModel.from_pretrained(model_path)


# UMAP model for dimensionality reduction
umap_model = UMAP(n_neighbors=30, n_components=5, min_dist=0.0, metric='cosine')

# HDBSCAN model for clustering
hdbscan_model = HDBSCAN(min_samples=20, 
                        gen_min_span_tree=True, 
                        prediction_data=True, 
                        min_cluster_size=5, 
                        metric='euclidean', 
                        cluster_selection_method='leaf')

# Create the CountVectorizer instance with the custom LemmaTokenizer
vectorizer_model = CountVectorizer(strip_accents='unicode', 
                                   stop_words='english', 
                                   ngram_range=(1, 3), 
                                   max_df=0.6) # remove general terms 
                                

# Step 1: Initialize custom c-TF-IDF model
ctfidf_model = ClassTfidfTransformer(
    bm25_weighting=False,             # BM25: Beyond a certain point, additional occurrences of a term don’t contribute as much to its weight, introducing a saturation effect. This is controlled by the k1 parameter.
    reduce_frequent_words=True,       # Reduce the impact of overly frequent words (True if needed)
)


# chain model: first extract the most relevant with KeyBERT, then prioritize diversity 
representation_model = [KeyBERTInspired(top_n_words=30, random_state=42), MaximalMarginalRelevance(diversity=0.7)]

# Initialize and fit BERTopic with probability calculation
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    ctfidf_model=ctfidf_model, 
    vectorizer_model=vectorizer_model,
    verbose=True,
    representation_model=representation_model,
    top_n_words=10,
    calculate_probabilities=True,  # Ensure that BERTopic calculates probabilities
)

# Fit the model with documents and embeddings
# probabilities 
topics, probabilities = topic_model.fit_transform(docs, embeddings=embeddings)

# Create the directory if it does not exist
os.makedirs('../../models/BERTopic_All_Pneumonia_Type', exist_ok=True)
os.makedirs('../../models/BERTopic_All_Pneumonia_Type/representations/', exist_ok=True)
os.makedirs('../../models/BERTopic_All_Pneumonia_Type/clusters/', exist_ok=True)

# Save the BERTopic model for later use as a directory 
# Save the ctf-idf matrix  for terms 
topic_model.save("../../models/BERTopic_All_Pneumonia_Type/representations/specter_representation_keybert_mmr", serialization="pytorch", save_ctfidf=True)

# Get HDBSCAN cluster labels and probabilities
hdbscan_labels = topic_model.hdbscan_model.labels_
#The prediction_data=True enables the model to calculate probabilities by predicting how confidently each document fits into a cluster (fits and transforms)
hdbscan_probabilities = topic_model.hdbscan_model.probabilities_ # These probabilities tell you how confidently HDBSCAN assigns a document to its cluster (after fitting embeddings according to hdb model)

# Save HDBSCAN cluster labels and probabilities
torch.save(hdbscan_labels, '../../models/BERTopic_All_Pneumonia_Type/clusters/hdbscan_labels_keybert_mmr.pt')
torch.save(hdbscan_probabilities, '../../models/BERTopic_All_Pneumonia_Type/clusters/hdbscan_probabilities_keybert_mmr.pt')

## Manually save some files to the model directory 
# Save BERTopic probabilities (topic-patent / document-topic matrix) inside the model directory
torch.save(probabilities, '../../models/BERTopic_All_Pneumonia_Type/representations/specter_representation_keybert_mmr/specter_probabilities_keybert_mmr.pt') # the actual document-topic probabilities that delineate association of each document to topic(s)

# Print the topic information after applying custom c-TF-IDF
topic_info = topic_model.get_topic_info()
torch.save(topic_info, '../../models/BERTopic_All_Pneumonia_Type/representations/specter_representation_keybert_mmr/topic_info_keybert_mmr.pt')
topic_info.to_csv('../../models/BERTopic_All_Pneumonia_Type/representations/specter_representation_keybert_mmr/topic_info.csv')
topic_info.to_excel('../../models/BERTopic_All_Pneumonia_Type/representations/specter_representation_keybert_mmr/topic_info.xlsx')

# Save the representation and count vectorizer models to a .pt file
torch.save(representation_model, '../../models/BERTopic_All_Pneumonia_Type/representations/specter_representation_keybert_mmr/representation_model.pt')
torch.save(vectorizer_model, '../../models/BERTopic_All_Pneumonia_Type/representations/specter_representation_keybert_mmr/vectorizer_model.pt')

Embeddings shape: (4442, 768)


2025-02-12 14:30:05,043 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-02-12 14:30:18,334 - BERTopic - Dimensionality - Completed ✓
2025-02-12 14:30:18,335 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-02-12 14:30:18,665 - BERTopic - Cluster - Completed ✓
2025-02-12 14:30:18,668 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-02-12 14:30:43,128 - BERTopic - Representation - Completed ✓


In [38]:
topic_model.visualize_topics() 